In [1]:
from datetime import datetime
import time

import numpy as np
import mlagents
from mlagents_envs.registry import default_registry
from mlagents_envs.environment import ActionTuple
from mlagents.trainers.settings import TorchSettings

from mlagents.torch_utils import torch, default_device, set_torch_config
from mlagents.plugins.ppo.PPO import PPO, RolloutBuffer

from torch.utils.tensorboard import SummaryWriter


In [2]:
set_torch_config(TorchSettings(device='cpu'))
print(default_device())

cpu


### Load a default scene
Load a default scene from the registry and reset it

In [3]:
env_id = '3DBall'

In [4]:
def hard_env_reset():
    try:
        env.close()
    except:
        pass

    return default_registry[env_id].make()

In [5]:
env = default_registry[env_id].make()#hard_env_reset()
env.reset()

In [6]:
env.close()

In [130]:
# collect trajectory only for agent 1 
done = False
behavior_name = list(env.behavior_specs)[0]
spec = env.behavior_specs[behavior_name]
env.reset()
step = 0
max_training_timesteps = 200
ppo_agent.buffer.clear()
while step < max_training_timesteps:
    
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    
    if 0 in terminal_steps.agent_id:
        done = True
        ppo_agent.buffer.rewards.append(terminal_steps[index].reward)
        ppo_agent.buffer.is_terminals.append(done)

        obs = terminal_steps[index].obs[0]
        action = ppo_agent.select_action(obs)

    env.set_actions(behavior_name, spec.action_spec.empty_action(len(decision_steps)))

    if 0 in decision_steps.agent_id:
        done = False

        ppo_agent.buffer.rewards.append(decision_steps[index].reward)
        ppo_agent.buffer.is_terminals.append(done)

        # get action
        obs = decision_steps[index].obs[0]
        action = ppo_agent.select_action(obs)
        action_tuple = ActionTuple()
        action_tuple.add_continuous(action[np.newaxis,:])

        env.set_action_for_agent(behavior_name, 0, action_tuple)

    if 0 in terminal_steps.agent_id and 0 in decision_steps.agent_id:
        print("wtf")

    env.step()
    step+=1
    print(step)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
wtf
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [49]:
state = terminal_steps[index].obs[0]
# select action with policy
action = ppo_agent.select_action(state)
action_tuple = ActionTuple()
action_tuple.add_continuous(action)
# print(action)

0.1

In [47]:
has_continuous_action_space = True  # continuous action space; else discrete

max_ep_len = 2000                   # max timesteps in one episode
max_training_timesteps = int(3e6)   # break training loop if timeteps > max_training_timesteps

print_freq = max_ep_len /10        # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len          # log avg reward in the interval (in num timesteps)
save_model_freq = int(1e5)          # save model frequency (in num timesteps)

action_std = 0.6                    # starting std for action distribution (Multivariate Normal)
action_std_decay_rate = 0.05        # linearly decay action_std (action_std = action_std - action_std_decay_rate)
min_action_std = 0.1                # minimum action_std (stop decay after action_std <= min_action_std)
action_std_decay_freq = int(2.5e5)  # action_std decay frequency (in num timesteps)

update_timestep = max_ep_len      # update policy every n timesteps
K_epochs = 80               # update policy for K epochs in one PPO update

eps_clip = 0.2          # clip parameter for PPO
gamma = 0.99            # discount factor

lr_actor = 0.0003       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 0         # set random seed if required (0 = no random seed)

In [48]:
def train(env, options):

    env.reset()
    
    # get the first behavior name and its spec
    behavior_name = list(env.behavior_specs)[0]
    spec = env.behavior_specs[behavior_name]

    # state space dimension
    state_dim = spec.observation_specs[0].shape[0]

    # action space dimension
    action_dim = len(spec.action_spec)

    # setup logger
    t = time.localtime()
    log_dir = './runs/3DBall/first_logs_{}/'.format(time.strftime("%d_%M_%Y-%H_%M", t))
    writer = SummaryWriter(log_dir)


    run_num_pretrained = 0      #### change this to prevent overwriting weights in same env_name folder

    hyperparam_log = """
    max training timesteps : {}, max timesteps per episode : {}, model saving frequency : {}, 
    state space dimension : {}, action space dimension : {}, action_std : {}, action decay rate {}, 
    min action std : {}, decay frequency : {}, PPO update freq : {}, PPO K epochs : {},
     Epsilon clip : {}, gamma : {}, Lr actor : {}, Lr Critic : {}
    """.format(max_training_timesteps, max_ep_len, save_model_freq, state_dim, action_dim,  action_std, action_std_decay_rate, min_action_std, action_std_decay_freq, update_timestep, K_epochs, eps_clip, gamma, lr_actor, lr_critic)
    
    writer.add_text("Hyperparameter", hyperparam_log)

    # initialize a PPO agent
    ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, has_continuous_action_space, action_std)


    # track total training time
    start_time = datetime.now().replace(microsecond=0)

    # printing and logging variables
    num_episode = 1
    logstep = 0
    running_reward =0 

    time_step = 0
    i_episode = 0
    
    # training loop
    while time_step <= max_training_timesteps:

        state = env.reset()
        current_ep_reward = 0
        print("time step:", time_step)

        #TODO: initialize with empty state
        for t in range(1, max_ep_len+1):
            
            decision_steps, terminal_steps = env.get_steps(behavior_name)

            # check if tracked agent has ended episode within step
            if 0 in terminal_steps.agent_id:
                done = True

                index = terminal_steps.agent_id_to_index[0]
                reward = terminal_steps[index].reward
                ppo_agent.buffer.rewards.append(reward)
                ppo_agent.buffer.is_terminals.append(done)

                obs = terminal_steps[index].obs[0]
                action = ppo_agent.select_action(obs)

                num_episode += 1

            env.set_actions(behavior_name, spec.action_spec.empty_action(len(decision_steps)))

            # check if tracked agent requires a new decision within step
            if 0 in decision_steps.agent_id:
                done = False

                index = decision_steps.agent_id_to_index[0]
                reward = decision_steps[index].reward
                ppo_agent.buffer.rewards.append(reward)
                ppo_agent.buffer.is_terminals.append(done)

                # get action
                obs = decision_steps[index].obs[0]
                action = ppo_agent.select_action(obs)
                action_tuple = ActionTuple()
                action_tuple.add_continuous(action[np.newaxis,:])

                env.set_action_for_agent(behavior_name, 0, action_tuple)


            env.step()

            time_step +=1
            current_ep_reward += reward

            # update PPO agent
            if time_step % update_timestep == 0:
                print("Update PPO")
                ppo_agent.update()

            # if continuous action space; then decay action std of ouput action distribution
            if has_continuous_action_space and time_step % action_std_decay_freq == 0:
                ppo_agent.decay_action_std(action_std_decay_rate, min_action_std)

            # log in logging file
            if time_step % print_freq == 0:
                writer.add_scalar("Reward per timestep", current_ep_reward/t ,logstep)
                logstep += 1
                

            # printing average reward
            if time_step % print_freq == 0:

 
                print("Episode : {} \t Timestep : {} \t Current Ep Reward : {} \t Average Reward : {}".format(i_episode, time_step, current_ep_reward, current_ep_reward/num_episode))

                
        writer.add_scalar("Reward per Episode", current_ep_reward ,i_episode)
        print("End of Episode : {} \t episode reward: {}".format(i_episode, current_ep_reward) )
        running_reward += current_ep_reward
        
        num_episode = 0
        i_episode += 1

    # env.close()


    # print total training time
    end_time = datetime.now().replace(microsecond=0)
    print("Started training at (GMT) : ", start_time)
    print("Finished training at (GMT) : ", end_time)
    print("Total training time  : ", end_time - start_time)
  




In [45]:
train(env, 0)

cpu
cpu
cpu
time step: 0
Episode : 0 	 Timestep : 10 	 Current Ep Reward : 1.0000000149011612 	 Average Reward : 1.0000000149011612
Episode : 0 	 Timestep : 20 	 Current Ep Reward : 0.9000000283122063 	 Average Reward : 0.45000001415610313
Episode : 0 	 Timestep : 30 	 Current Ep Reward : 1.9000000432133675 	 Average Reward : 0.9500000216066837
Episode : 0 	 Timestep : 40 	 Current Ep Reward : 1.8000000566244125 	 Average Reward : 0.6000000188748041
Episode : 0 	 Timestep : 50 	 Current Ep Reward : 2.8000000715255737 	 Average Reward : 0.9333333571751913
Episode : 0 	 Timestep : 60 	 Current Ep Reward : 3.800000086426735 	 Average Reward : 1.2666666954755783
Episode : 0 	 Timestep : 70 	 Current Ep Reward : 4.800000101327896 	 Average Reward : 1.6000000337759654
Episode : 0 	 Timestep : 80 	 Current Ep Reward : 4.700000114738941 	 Average Reward : 1.1750000286847353
Episode : 0 	 Timestep : 90 	 Current Ep Reward : 5.700000129640102 	 Average Reward : 1.4250000324100256
Update PPO
Epis

KeyboardInterrupt: 

In [46]:
env = hard_env_reset()